## Домашнее творческое задание по дисциплине "Технологии обработки больших данных" (часть 2)

Выполнил: студент группы __ПМ20-1 Джаводов Санджар Далерович__ и студент группы __ПМ20-4 Кахоров Пайрав Анварович__

Вариант: __27__

    

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

__В задачах, связанных с пакетом `dask`, явно указано, какой коллекцией вы должны пользоваться при решении задачи. Все операции вычислений должны проводиться  над этой коллекцией и средствами пакета `dask`, если в задании не сказано обратного.__ 



__Задание 1__. В файле `scores.npy` содержится массив оценок рейтингов вопросов. Напишите функцию, которая считает сумму квадратов цифр числа. Векторизуйте эту функцию при помощи `numba.vectorize` и посчитайте количество чисел из файла, сумма квадратов цифр которых меньше 50.

In [2]:
import dask.array as da 
import numpy as np
from numba import vectorize, int64, float32, float64, int32
import pandas as pd
from multiprocessing import Pool
import multiprocessing as mp
import h5py
import dask.bag as db
import json
import dask.dataframe as dd
import sqlite3
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
x = np.load('scores.npy')

In [3]:
@vectorize([int64(int64)])
def f(x):
    x = abs(x)
    return (x//1000)**2 + (x//100%10)**2 + (x%100//10)**2 +(x%10)**2 < 50

In [4]:
sum(f(x))

594575

__Задание 2__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Создайте базу данных SQLite с таблицей `Questions`. Работая с объектом-курсором, выполните запрос на языке SQL, который возращает вопросы, касающиеся фреймворка Django.

In [108]:
df = pd.read_csv('questions.csv').iloc[:,-2:]

In [97]:
con = sqlite3.connect('ques.db')

In [98]:
cur = con.cursor()

In [99]:
cur.execute('Create table Questions(Title, Body);')

In [103]:
data = list(df[['Title', 'Body']].itertuples(index = False, name = None))


In [104]:
cur.executemany("Insert Into Questions Values(?,?)", data)

In [105]:
res = cur.execute("""Select * 
                    from Questions 
                    where Title Like '%django%' 
                    OR Title Like '%Django%'
                    OR Body Like '%django%'
                    OR Body Like '%Django%'""")
res.fetchmany()

[('Class views in Django',
  '<p><a href="http://www.djangoproject.com/">Django</a> view points to a function, which can be a problem if you want to change only a bit of functionality. Yes, I could have million keyword arguments and even more if statements in the function, but I was thinking more of an object oriented approach.</p>\r\\n\r\\n<p>For example, I have a page that displays a user. This page is very similar to page that displays a group, but it\'s still not so similar to just use another data model. Group also has members etc...</p>\r\\n\r\\n<p>One way would be to point views to class methods and then extend that class. Has anyone tried this approach or has any other idea? </p>')]

In [106]:
len(res.fetchall())

61596

__Задание 3__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow. Используя `multiprocessing.Pool` выясните, сколько вопросов задали в 2011 году. При решении задачи разделите файл на части, для обработки каждой из частей используйте параллельные вычисления.

In [3]:
df = pd.read_csv('questions.csv')

In [4]:
df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [14]:
%%file date.py

def date(x):
    s = 0
    if x[:4] == '2011':
        s += 1
    return s

Overwriting date.py


In [15]:
import date

In [16]:
pool = mp.Pool()
results = pool.map(date.date,list(df['CreationDate']))
print(sum(results))

40910


__Задание 4__. В файле `qembeddings.hdf` находятся векторные представления заголовков вопросов, заданные на сайте stackoverflow. В файле `qtitles.pickle` находится список заголовков этих вопросов. Напишите функцию, которая для вопроса с индексом `i` возвращает количество вопросов, находящихся от него на расстоянии не более `k` (`k` - параметр функции). Выберите метрику, наиболее подходящую (на ваш взгляд) для решения этой задачи. Для работы с эмбеддингами используйте `dask.array`.

In [2]:
qembeddings = h5py.File('qembeddinbgs.h5','r')

In [3]:
list(qembeddings.keys())

['embeddings']

In [4]:
xdata = qembeddings.get('embeddings')
xdata= np.array(xdata)

In [117]:
arr = da.from_array(xdata, chunks = 'auto')
arr

dask.array<array, shape=(600000, 312), dtype=float32, chunksize=(107546, 312), chunktype=numpy.ndarray>

In [ ]:
def evklid(i,k): 
    a = dataset1[i] 
    lst = [] 
    for j in dataset1: 
        b = math.sqrt(sum((j - a)**2)) 
        if b <=k: 
            lst.append(b) 
    return lst 


len(evklid(10,0.5))

__Задание 5__. В файлах `questions_with_answers_*.jsonl` содержатся вопросы, заданные на сайте stackoverflow, и ответы на них. Посчитайте и выведите на экран количество вопросов, на которые дали более 10 ответов. Для обработки данных используйте `dask.bag`.

In [5]:
qs = db.read_text('questions_with_answers/*')

In [6]:
qs = qs.map(json.loads)

In [94]:
def answer_score(d):
    count = []
    for i in d['answers']:
        count.append(i['Score'])
    if sum(count) >= 10:
        return True
    else:
        return False
        

In [96]:
sum(qs.map(answer_score).compute())

32893

__Задание 6__. В файле `questions.csv` содержатся вопросы, заданные на сайте stackoverflow.Проанализируйте и сделайте вывод: существует ли корреляция между длиной описания вопроса и оценкой score этого вопроса. Для обработки данных используйте `dask.dataframe`.

In [45]:
df_dask = dd.read_csv('questions.csv')  

In [72]:
s1 = df_dask['Body'].map(lambda x: len(x)).compute()

In [73]:
s2 = df_dask['Score'].compute()

In [77]:
s1.corr(s2)

-0.022158584606227983